In [14]:
import time
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pulp as pl
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")

In [20]:
# ----------------------------------------------------------------------
# Helper de download
# ----------------------------------------------------------------------
def _download_with_retry(ticker, start, end, attempts=3):
    for _ in range(attempts):
        try:
            df = yf.download(
                ticker, start=start, end=end,
                progress=False, auto_adjust=False, threads=False
            )
            if not df.empty:
                return df
        except Exception:
            pass
        time.sleep(1)
    return pd.DataFrame()

In [21]:
# ----------------------------------------------------------------------
# Classe principal
# ----------------------------------------------------------------------
class PortfolioSimulator:
    def __init__(self, df_portfolio, valor_aporte_mensal=7_250.0, cdi_aa=0.13):
        self.df_original = df_portfolio.copy()
        if self.df_original["% Ideal - Ref."].max() > 1:
            self.df_original["% Ideal - Ref."] /= 100.0
        self.aporte_mensal = valor_aporte_mensal
        self.cdi_aa = cdi_aa
        # ← lista de classes p/ cálculo de drift
        self.classes = sorted(self.df_original["Classe"].unique())
        
        # Nova estrutura para armazenar aportes detalhados
        self.aportes_detalhados = []

    # ------------- mapeamento de tickers -----------------
    def mapear_tickers(self):
        """Apenas ativos negociados via Yahoo; RF especial fica com próprio nome."""
        mapa = {}
        for _, row in self.df_original.iterrows():
            tk, geo = row["Ticker"], row["Geo."]
            if tk in {"SELIC", "FDI", "FRFH", "LC", "CDB",
                    "IPCA", "CDBI", "PRE", "PGBL"}:
                mapa[tk] = None          # marcador especial
            elif geo == "BR":
                mapa[tk] = f"{tk}.SA"
            elif geo == "US":
                mapa[tk] = tk
        return mapa
    
    @staticmethod
    def _selic_fator_mensal(start, end):
        """SGS 4390 Selic diária → fator acumulado em datas 'M' (últ. dia útil do mês)."""
        url = ("https://api.bcb.gov.br/dados/serie/bcdata.sgs.4390/dados"
            f"?formato=json&dataInicial={start:%d/%m/%Y}&dataFinal={end:%d/%m/%Y}")
        js  = requests.get(url, timeout=10).json()
        df  = pd.DataFrame(js)
        df["data"]  = pd.to_datetime(df["data"], format="%d/%m/%Y")
        df["valor"] = df["valor"].astype(float) / 100        # diário (fração)
        df["fator"] = (1 + df["valor"]).cumprod()
        fator_m = df.set_index("data").resample("M").last()["fator"]
        return fator_m

    @staticmethod
    def _ipca_fator_mensal(start, end):
        """SGS 433 IPCA % mensal → fator acumulado (base 1)."""
        url = ("https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados"
            f"?formato=json&dataInicial={start:%d/%m/%Y}&dataFinal={end:%d/%m/%Y}")
        js  = requests.get(url, timeout=10).json()
        df  = pd.DataFrame(js)
        df["data"]  = pd.to_datetime(df["data"], format="%d/%m/%Y") + pd.offsets.MonthEnd(0)
        df["valor"] = df["valor"].astype(float) / 100
        df["fator"] = (1 + df["valor"]).cumprod()
        return df.set_index("data")["fator"]

    @staticmethod
    def _fator_mensal_fixo(start, end, taxa_anual):
        """Retorna série mensal de fator acumulado (base 1.0) para uma taxa 'anual' constante."""
        idx = pd.date_range(start, end, freq="M")
        taxa_mensal = (1 + taxa_anual) ** (1/12) - 1          # ⇐ conversão
        return pd.Series((1 + taxa_mensal) ** np.arange(1, len(idx)+1), index=idx)

    # ------------- históricos ----------------------------
    def obter_dados_historicos(self, meses):
        end_date   = datetime.now()
        start_date = end_date - timedelta(days=meses*30)

        dados, mapa = {}, self.mapear_tickers()

        usd = _download_with_retry("USDBRL=X", start_date, end_date)["Adj Close"]
        if usd.empty:
            raise RuntimeError("Falha USD/BRL")

        print("Coletando séries…")
        for tk, sym in mapa.items():
            # ─── Renda-fixa e índices ────────────────────────────────
            if tk in {"SELIC", "FDI", "FRFH", "LC", "CDB"}:
                dados[tk] = self._selic_fator_mensal(start_date, end_date)
                # print(f"✓ Selic {tk}")
                continue
            if tk in {"IPCA", "CDBI"}:
                dados[tk] = self._ipca_fator_mensal(start_date, end_date)
                # print(f"✓ IPCA  {tk}")
                continue
            if tk == "PRE":
                dados[tk] = self._fator_mensal_fixo(start_date, end_date, 0.09)   # 9 % a.a.
                # print("✓ PRE  (9 % a.a.)")
                continue
            if tk == "PGBL":
                dados[tk] = self._fator_mensal_fixo(start_date, end_date, 0.07)   # 7 % a.a.
                # print("✓ PGBL (7 % a.a.)")
                continue

            # ─── Ações / ETFs ───────────────────────────────────────
            dfp = _download_with_retry(sym, start_date, end_date)
            if dfp.empty:
                print(f"⚠️  Sem dados {tk}")
                continue
            serie_m = dfp["Adj Close"].resample("M").last()
            if self.df_original.loc[self.df_original["Ticker"] == tk, "Geo."].iat[0] == "US":
                usd_m = usd.resample("M").last()
                serie_m.values[:] = serie_m.to_numpy()*usd_m.to_numpy()

            dados[tk] = serie_m
            # print(f"✓ {tk}")

        return dados

    # ------------- estratégia 1 -------------------------
    def _aporte_deficit(self, df, valor_cart, aporte, mes, data):
        df = df[df["Cotação"].notna() & np.isfinite(df["Cotação"])].copy()
        df["Valor Ideal"] = df["% Ideal - Ref."] * (valor_cart + aporte)
        df["deficit"] = (df["Valor Ideal"] - df["Total"]).clip(lower=0)
        
        if df["deficit"].sum() == 0:
            return df, aporte
            
        df["aporte_sugerido"] = df["deficit"] / df["deficit"].sum() * aporte
        df["Qtd_comprar"], df["Custo_real"] = 0.0, 0.0
        
        for i, r in df.iterrows():
            qtd = (r["aporte_sugerido"] / r["Cotação"]) if (r["Classe"] == "RF" and r["Ticker"] != "IMAB11") \
                  else np.floor(r["aporte_sugerido"] / r["Cotação"])
            df.at[i, "Qtd_comprar"] = qtd
            df.at[i, "Custo_real"]  = qtd * r["Cotação"]
            
            # Salvar detalhes do aporte se houve compra
            if qtd > 0:
                self._salvar_aporte_detalhado(r, qtd, qtd * r["Cotação"], "Deficit", mes, data, df)

        return df, aporte - df["Custo_real"].sum()

    # ------------- estratégia 2 -------------------------
    def _aporte_po(self, df, valor_cart, aporte, mes, data):
        df = df[df["Cotação"].notna() & np.isfinite(df["Cotação"])].copy()
        if df.empty:
            return self._aporte_deficit(df, valor_cart, aporte, mes, data)

        df["Valor Ideal"] = df["% Ideal - Ref."] * (valor_cart + aporte)
        df["deficit"] = (df["Valor Ideal"] - df["Total"]).clip(lower=0)
        prob, qt_rf, qt_rv, gap = pl.LpProblem("PO", pl.LpMinimize), {}, {}, {}

        for idx, r in df.iterrows():
            nome = str(r["Ativo"]).replace(" ", "_").replace(".", "_")
            if (r["Classe"] == "RF" and r["Ticker"] != "IMAB11"):
                qt_rf[idx] = pl.LpVariable(f"RF_{nome}", lowBound=0)
            else:
                qt_rv[idx] = pl.LpVariable(f"RV_{nome}", lowBound=0, cat="Integer")
            gap[idx] = pl.LpVariable(f"GAP_{nome}", lowBound=0)

        prob += pl.lpSum(gap.values())
        prob += (pl.lpSum(df.loc[i,"Cotação"]*qt_rf[i] for i in qt_rf) +
                 pl.lpSum(df.loc[i,"Cotação"]*qt_rv[i] for i in qt_rv)) <= aporte

        for idx in df.index:
            preco = df.at[idx, "Cotação"]
            compra = preco * (qt_rf[idx] if idx in qt_rf else qt_rv[idx])
            prob += gap[idx] >= df.at[idx,"deficit"] - compra

        try:
            prob.solve(pl.PULP_CBC_CMD(msg=0))
        except pl.PulpError:
            prob.solve()

        if prob.status != pl.LpStatusOptimal:
            print(f'Solução com LP não convergiu')
            return self._aporte_deficit(df, valor_cart, aporte, mes, data)

        df["Qtd_comprar"], df["Custo_real"] = 0.0, 0.0
        
        for idx, var in qt_rf.items():
            qtd = var.varValue or 0
            df.at[idx,"Qtd_comprar"] = qtd
            df.at[idx,"Custo_real"]  = qtd * df.at[idx,"Cotação"]
            
            # Salvar detalhes do aporte se houve compra
            if qtd > 0:
                self._salvar_aporte_detalhado(df.loc[idx], qtd, qtd * df.at[idx,"Cotação"], "PO", mes, data, df)
                
        for idx, var in qt_rv.items():
            qtd = int(var.varValue or 0)
            df.at[idx,"Qtd_comprar"] = qtd
            df.at[idx,"Custo_real"]  = qtd * df.at[idx,"Cotação"]
            
            # Salvar detalhes do aporte se houve compra
            if qtd > 0:
                self._salvar_aporte_detalhado(df.loc[idx], qtd, qtd * df.at[idx,"Cotação"], "PO", mes, data, df)

        return df, aporte - df["Custo_real"].sum()

    # ------------- método para salvar aportes detalhados ---------------
    def _salvar_aporte_detalhado(self, row, qtd_aportada, valor_aportado, estrategia, mes, data, df_cart):
        """Salva os detalhes do aporte para um ativo específico"""
        # Calcular % Atual e Variação
        valor_total_carteira = df_cart["Total"].sum()
        pct_atual = (row["Total"] / valor_total_carteira) * 100 if valor_total_carteira > 0 else 0
        pct_ideal = row["% Ideal - Ref."] * 100
        variacao = pct_atual - pct_ideal
        
        aporte_info = {
            'Mes': mes,
            'Data': data,
            'Estrategia': estrategia,
            'Geo': row.get("Geo.", ""),
            'Classe': row.get("Classe", ""),
            'Subclasses': row.get("Subclasses", ""),
            'Setor': row.get("Setor", ""),
            'Ativo': row.get("Ativo", ""),
            'Ticker': row.get("Ticker", ""),
            'Qnt_Total': row.get("Qnt.", 0),
            'Cotacao': row.get("Cotação", 0),
            'Total': row.get("Total", 0),
            'Pct_Atual': pct_atual,
            'Pct_Ideal': pct_ideal,
            'Variacao': variacao,
            'Qnt_Aportado': qtd_aportada,
            'Valor_Aportado': valor_aportado
        }
        
        self.aportes_detalhados.append(aporte_info)

    # ------------- método para obter dataframe de aportes --------------
    def obter_df_aportes(self):
        """Retorna DataFrame com todos os aportes detalhados"""
        if not self.aportes_detalhados:
            return pd.DataFrame()
        
        df_aportes = pd.DataFrame(self.aportes_detalhados)
        
        # Renomear colunas para o formato solicitado
        # df_aportes = df_aportes.rename(columns={
        #     'Qnt_Total': 'Qnt',
        #     'Cotacao': 'Cotação',
        #     'Pct_Atual': '% Atual',
        #     'Pct_Ideal': '% Ideal',
        #     'Qnt_Aportado': 'Qnt aportado',
        #     'Valor_Aportado': 'Valor aportado'
        # })
        
        # Separar por estratégia
        # df_deficit = df_aportes[df_aportes['Estrategia'] == 'Deficit'].copy()
        # df_po = df_aportes[df_aportes['Estrategia'] == 'PO'].copy()
        
        # if not df_deficit.empty:
        #     df_deficit = df_deficit.rename(columns={
        #         'Qnt_Aportado': 'Qnt_aportado_deficit',
        #         'Valor_Aportado': 'Valor_aportado_deficit'
        #     })
            
        # if not df_po.empty:
        #     df_po = df_po.rename(columns={
        #         'Qnt_Aportad': 'Qnt aportado PO',
        #         'Valor_Aportad': 'Valor aportado PO'
        #     })
        
        # return df_deficit, df_po
        return df_aportes

    # ------------- loop principal -----------------------
    def simular(self, meses=24):
        # Limpar aportes anteriores
        self.aportes_detalhados = []
        
        dados = self.obter_dados_historicos(meses)
        cart_d, cart_p = self.df_original.copy(), self.df_original.copy()

        dates = pd.date_range(end=datetime.now(), periods=meses, freq="MS")
        valor_inicial = cart_d["Total"].sum()
        # valor_inicial_classe = (
        #     cart_d.groupby("Classe")["Total"].sum().to_dict()
        # )

        aporte_acum, out = 0.0, []

        # --------------------------------------------------------------
        # acumuladores de aportes por classe e por estratégia
        # --------------------------------------------------------------
        aportado_def = {cls: 0.0 for cls in self.classes}
        aportado_po  = {cls: 0.0 for cls in self.classes}

        for imes, dt in enumerate(dates, 1):
            # print(imes)
            # preços atualizados
            for cart in (cart_d, cart_p):
                for idx, r in cart.iterrows():
                    tk = r["Ticker"]
                    if tk not in dados: continue
                    s = dados[tk][dados[tk].index <= dt]
                    
                    if s.empty:
                        continue
                    v_raw = s.iloc[-1]                     # pode ser escalar ou Series
                    # extrai o primeiro elemento se for iterável
                    v = float(np.asarray(v_raw).flatten()[0])
                    cart.at[idx, "Cotação"] = v
                cart["Total"] = cart["Qnt."] * cart["Cotação"]

            # aportes (passando mes e data para tracking)
            aporte_acum += self.aporte_mensal
            res_d, sobra_d = self._aporte_deficit(cart_d, cart_d["Total"].sum(), self.aporte_mensal, imes, dt)
            res_p, sobra_p = self._aporte_po(cart_p, cart_p["Total"].sum(), self.aporte_mensal, imes, dt)

            cart_d["Qnt."] += res_d["Qtd_comprar"]; cart_p["Qnt."] += res_p["Qtd_comprar"]
            cart_d["Total"] = cart_d["Qnt."] * cart_d["Cotação"]
            cart_p["Total"] = cart_p["Qnt."] * cart_p["Cotação"]

            # sobras → SELIC
            for cart, sobra, estrategia in ((cart_d,sobra_d,"Deficit"),(cart_p,sobra_p,"PO")):
                if sobra>0 and (cart["Ticker"]=="SELIC").any():
                    idx = cart[cart["Ticker"]=="SELIC"].index[0]
                    cart.at[idx,"Qnt."]  += sobra
                    cart.at[idx,"Total"] += sobra
                    
                    # Registrar sobra como aporte na SELIC
                    if sobra > 0:
                        self._salvar_aporte_detalhado(cart.loc[idx], sobra, sobra, estrategia, imes, dt, cart)

            investido = valor_inicial + aporte_acum
            vt_d, vt_p = cart_d["Total"].sum(), cart_p["Total"].sum()
            rent_def_corr = (vt_d/investido -1)*100
            rent_po_corr = (vt_p/investido -1)*100
            # total atual por classe
            # totais_def = cart_d.groupby("Classe")["Total"].sum()
            # totais_po  = cart_p.groupby("Classe")["Total"].sum()
            # rentabilidade % por classe (RF e RV)
            # rent_def_rf = (
            #     (totais_def.get("RF", 0) /
            #     valor_inicial_classe.get("RF", 1) - 1) * 100
            # )
            # rent_def_rv = (
            #     (totais_def.get("RV", 0) /
            #     valor_inicial_classe.get("RV", 1) - 1) * 100
            # )
            # rent_po_rf  = (
            #     (totais_po.get("RF", 0) /
            #     valor_inicial_classe.get("RF", 1) - 1) * 100
            # )
            # rent_po_rv  = (
            #     (totais_po.get("RV", 0) /
            #     valor_inicial_classe.get("RV", 1) - 1) * 100
            # )

            # ---- drift por classe (usar carteira déficit para referência) ---
            drift = {}
            for cls in self.classes:
                # pesos alvo são os mesmos p/ as duas carteiras
                peso_alvo = cart_d[cart_d["Classe"] == cls]["% Ideal - Ref."].sum()

                peso_def  = cart_d[cart_d["Classe"] == cls]["Total"].sum() / vt_d
                peso_po   = cart_p[cart_p["Classe"] == cls]["Total"].sum() / vt_p

                drift[f"drift_{cls}_def"] = (peso_def - peso_alvo) * 100   # p.p.
                drift[f"drift_{cls}_po"]  = (peso_po  - peso_alvo) * 100

            out.append({
                "mes": imes, "data": dt,
                "investido": investido,
                "valor_def": vt_d, "valor_po": vt_p,
                "deficit_def": (cart_d["% Ideal - Ref."]*vt_d - cart_d["Total"]).clip(lower=0).sum(),
                "deficit_po":  (cart_p["% Ideal - Ref."]*vt_p - cart_p["Total"]).clip(lower=0).sum(),
                "rent_def_corr": rent_def_corr,
                "rent_po_corr":  rent_po_corr,
                # "rent_def_rf"  : rent_def_rf,
                # "rent_po_rf"   : rent_po_rf,
                # "rent_def_rv"  : rent_def_rv,
                # "rent_po_rv"   : rent_po_rv,
                **drift
            })

        return pd.DataFrame(out)

    # ------------- plotagem -----------------------------
    def plotar(self, df):
        cor_def, cor_po = "#060dd6", "#de1f35"
        pal_classes = ['forestgreen', 'black']  #descomentar quando trabalhar com classes

        fig, axs = plt.subplots(2, 2, figsize=(16, 11))
        (a1, a2), (a3, a4) = axs

        # 1) Déficit (log)
        eps = 1e-2
        sns.lineplot(x=df["mes"], y=df["deficit_def"].clip(lower=eps), ax=a1,
                    color=cor_def, label="Déficit", lw=2.5)
        sns.lineplot(x=df["mes"], y=df["deficit_po"].clip(lower=eps), ax=a1,
                    color=cor_po,  label="PO", lw=2.5)
        a1.set(title="Déficit Total (log)",
               xlabel="Mês", 
               ylabel="R$")
        a1.grid(ls="--", alpha=.5)

        # 2) Rentabilidade corrigida
        sns.lineplot(x="mes", y="rent_def_corr", data=df, ax=a2,
                    color=cor_def, label="Déficit",lw=2.5 )
        sns.lineplot(x="mes", y="rent_po_corr",  data=df, ax=a2,
                    color=cor_po,  label="PO", lw=2.5)
        a2.set(title="Rentabilidade (%) sobre capital investido",
            xlabel="Mês", ylabel="%")
        a2.grid(ls="--", alpha=.25)

        # 3) Drift por classe – duas curvas por classe
        for cls, cor in zip(self.classes, pal_classes):
            sns.lineplot(x="mes", y=f"drift_{cls}_def", data=df, ax=a3,
                        color=cor, lw=2.5,
                        label=f"{cls} – Déficit")
            sns.lineplot(x="mes", y=f"drift_{cls}_po",  data=df, ax=a3,
                        color=cor, lw=2.5, ls="--",
                        label=f"{cls} – PO")

        a3.axhline(0, ls=":", c="k", lw=1)
        a3.set(title="Drift por classe (p.p.)",
            xlabel="Mês", ylabel="Peso real – alvo")
        a3.grid(ls="--", alpha=.25)
        # legenda compacta: remove duplicatas mantendo ordem
        handles, labels = a3.get_legend_handles_labels()
        uniq = dict(zip(labels, handles))
        a3.legend(uniq.values(), uniq.keys(), ncol=2, frameon=False)

        # 4) Valor de mercado
        sns.lineplot(x="mes", y="valor_def", data=df, ax=a4,
                    color=cor_def, label="Déficit",lw=2.0 )
        sns.lineplot(x="mes", y="valor_po",  data=df, ax=a4,
                    color=cor_po,  label="PO",lw=2.0)
        a4.set(title="Valor da Carteira (R$)", xlabel="Mês", ylabel="R$")
        a4.grid(ls="--", alpha=.25)

        # eixo x inteiro
        for ax in axs.flat:
            ax.set_xticks(df["mes"])

        plt.tight_layout()
        plt.show()
        return fig

In [22]:
# ----------------------------------------------------------------------
# Execução
# ----------------------------------------------------------------------

df_port = pd.read_csv('./data/portfolio_20250526_tratado.csv', sep=';', index_col=0)
sim     = PortfolioSimulator(df_port, valor_aporte_mensal=7_250)

print("🚀 Simulando…")
df_out  = sim.simular(meses=24)

# print("📊 Plotando…")
# sim.plotar(df_out)

# 🆕 NOVO: Obter DataFrames de aportes detalhados
print("📋 Gerando DataFrames de aportes detalhados...")
# df_aportes_deficit, df_aportes_po = sim.obter_df_aportes()
df_aportes = sim.obter_df_aportes()

# print(f"✅ Aportes Déficit: {len(df_aportes_deficit)} registros")
# print(f"✅ Aportes PO: {len(df_aportes_po)} registros")
print("✅ Fim!")

🚀 Simulando…
Coletando séries…
📋 Gerando DataFrames de aportes detalhados...
✅ Fim!


In [12]:
df_aportes.head()

,Mes,Data,Estrategia,Geo,Classe,Subclasses,Setor,Ativo,Ticker,Qnt_Total,Cotacao,Total,Pct_Atual,Pct_Ideal,Variacao,Qnt_Aportado,Valor_Aportado
0,1,2023-07-01 16:16:41.999528,Deficit,BR,RF,Juros Pós,-,Tesouro SELIC,SELIC,12649.37,1.0107,12784.718259,6.744024,12.419206,-5.675182,757.792486,765.900866
1,1,2023-07-01 16:16:41.999528,Deficit,BR,RF,Juros Pós,-,Fundo DI Simples,FDI,0.00,1.0107,0.000000,0.000000,5.353106,-5.353106,684.810357,692.137828
2,1,2023-07-01 16:16:41.999528,Deficit,BR,RF,Juros Pós,-,Fundo Renda Fixa High Grade Crédito Privado,FRFH,0.00,1.0107,0.000000,0.000000,3.211864,-3.211864,410.886214,415.282697
3,1,2023-07-01 16:16:41.999528,Deficit,BR,RF,Inflação,-,Tesouro IPCA 2029,IPCA,9449.86,0.9992,9442.300112,4.980876,5.931242,-0.950365,146.718681,146.601306
4,1,2023-07-01 16:16:41.999528,Deficit,BR,RF,Inflação,-,Tesouro IPCA 2040,IPCA,0.00,0.9992,0.000000,0.000000,3.211864,-3.211864,415.615189,415.282697


In [8]:
sns.set_theme(style="whitegrid", context="paper") 